Ah, tôi hiểu vấn đề rồi. Khi gửi message history cho OpenAI API, chúng ta cần chuyển đổi roleA/roleB thành user/assistant tương ứng. Đây là cách sửa:

```python:6_TuningWith2Prompting/main.py
def convert_roles_for_api(messages, is_roleA_turn=True):
    """
    Chuyển đổi roleA/roleB thành user/assistant cho OpenAI API
    is_roleA_turn: True nếu đang là lượt của roleA, False nếu là lượt của roleB
    """
    converted_messages = []
    for msg in messages:
        if is_roleA_turn:
            # Khi là lượt của roleA
            role = "user" if msg["role"] == "roleA" else "assistant"
        else:
            # Khi là lượt của roleB
            role = "user" if msg["role"] == "roleB" else "assistant"
        
        converted_messages.append({
            "role": role,
            "content": msg["content"]
        })
    return converted_messages

def simulate_conversation(row):
    """Simulate a conversation based on the row of data."""
    message_history = []
    response_times = []
    conversationTurnCount = 0

    # Extract settings from Excel row
    roleA_prompt = row['roleA_prompt']
    roleB_prompt = row['roleB_prompt']
    initialConversationHistory = row['initialConversationHistory']
    maxTurns = row['maxTurns']

    # Parse and add conversation history if exists
    history = parse_conversation_history(initialConversationHistory)
    if history:
        message_history.extend(history)
        print("\n=== Initial Conversation History ===")
        for msg in history:
            print(f"{msg['role']}: {msg['content']}")
    
    # Start conversation loop
    while conversationTurnCount < maxTurns:
        try:
            # Generate roleA message
            start_time = time.time()
            api_messages = [{"role": "system", "content": roleA_prompt}]
            # Chuyển đổi history cho roleA
            if message_history:
                converted_history = convert_roles_for_api(message_history, is_roleA_turn=True)
                api_messages.extend(converted_history)
            
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=api_messages,
                temperature=0.3
            )
            end_time = time.time()
            roleA_message = response.choices[0].message.content
            message_history.append({"role": "roleA", "content": roleA_message})
            response_times.append(end_time - start_time)
            print(f"\nRoleA: {roleA_message}")
            
            # Generate roleB response
            start_time = time.time()
            api_messages = [{"role": "system", "content": roleB_prompt}]
            # Chuyển đổi history cho roleB
            if message_history:
                converted_history = convert_roles_for_api(message_history, is_roleA_turn=False)
                api_messages.extend(converted_history)
            
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=api_messages,
                temperature=0
            )
            end_time = time.time()
            roleB_message = response.choices[0].message.content
            message_history.append({"role": "roleB", "content": roleB_message})
            response_times.append(end_time - start_time)
            print(f"RoleB: {roleB_message}")
            
            conversationTurnCount += 1
            time.sleep(1)

        except Exception as e:
            print(f"Error during conversation: {str(e)}")
            break

    return message_history, response_times
```

Những thay đổi chính:

1. Thêm hàm `convert_roles_for_api` để chuyển đổi role:
   - Khi là lượt của roleA:
     - roleA -> user
     - roleB -> assistant
   - Khi là lượt của roleB:
     - roleB -> user
     - roleA -> assistant

2. Áp dụng chuyển đổi role trước khi gọi API:
   - Tạo api_messages riêng cho mỗi lượt
   - Chuyển đổi history phù hợp với từng role

3. Giữ nguyên format roleA/roleB trong message_history để lưu trữ

Cách này sẽ giúp:
- OpenAI API hiểu đúng context của cuộc hội thoại
- Duy trì tính nhất quán của các role trong hội thoại
- Tránh việc AI bị nhầm lẫn về vai trò của mình
